# yeah, was trying to combine all cells and do pca, but seems a bit too tricky for me - JSB

In [ ]:
# all modules necessary for this nb
import os
import sys
import pickle

import numpy as np
np.random.seed(0)
import pylab as pl
import scipy.stats as sstats

# setting parameters for default matplotlib plots
%matplotlib inline

In [ ]:
pl.style.use('fabiostyle')
# needs to find the library of functions
sys.path.append('../../../code/')  # to be replaced!

import utils as ut
import plots as pt

In [ ]:
NOTEBOOK_NAME = 'pca_prepost'

In [ ]:
from pickleshare import PickleShareDB

autorestore_folder = os.path.join(os.getcwd(), 'autorestore', NOTEBOOK_NAME)
db = PickleShareDB(autorestore_folder)
import sys
from workspace import *
import IPython
ip = IPython.get_ipython()

# this will restore all the saved variables. ignore the errors listed.
load_workspace(ip, db)

# use `save_worspace(db)` to save variables at the end

In [ ]:
#from itertools import product
#def load_all_animals_vars(varname, animal_list, pilot_list=None, notebook='preprocessing'):
#    all_of_them = {}
#    for ani in animal_list:
#        folder = '../%s/notebooks/autorestore/%s/' % (ani, notebook)
#        all_of_them.update({ani: ut.load_variable(varname, folder)})
#    return all_of_them

In [ ]:
from itertools import product
def load_all_animals_vars(varname, animal_list, pilot_list=None, notebook='preprocessing', session='',
                          func=lambda x: x):
    all_of_them = {}
    for ani in animal_list:
        try:
            folder = '../%s/%s/notebooks/autorestore/%s/' % (ani, session, notebook)
            all_of_them.update({ani: func(ut.load_variable(varname, folder))})
        except IOError:
            print "Nope: %s, %s, %s" % (ani, session, varname)
    return all_of_them

In [ ]:
pre_animals  = 'c3m1', 'c3m2', 'c3f3','c10m8'
post_animals = '90','90_475','91','93_500','93_580','93_720','93_810','c3m1', 'c3m2', 'c3f3', 'c10m8'

In [ ]:
#labels = load_all_animals_vars('labels', all_animals, notebook='decoding_four_odors', session='pre')

In [ ]:
variables = ('time_ax', 'time_ax_single', 'events', 'cycles',
            'is_STIM1t', 'is_STIM2t', 'is_STIM3t', 'is_STIM4t',
            'traces',
            'STIM1_ons', 'STIM2_ons', 'STIM3_ons', 'STIM4_ons',
            'CYCLE_START', 'CS_DURATION')
vars_pre={}
for var in variables:
    vars_pre[var] = load_all_animals_vars(var, pre_animals, notebook='preprocessing', session='pre')

vars_post={}
for var in variables:
    vars_post[var] = load_all_animals_vars(var, post_animals, notebook='preprocessing', session='post')

#vars_pre = {}
#folder = '../pre/notebooks/autorestore/preprocessing/'
#for var in variables: 
#    vars_pre.update({var:ut.load_variable(var, folder=folder)})
#    
#vars_post = {}
#folder = '../post/notebooks/autorestore/preprocessing'
#for var in variables:
#    vars_post.update({var:ut.load_variable(var, folder=folder)})

In [ ]:
vars_pre['traces']['c3m1']

In [ ]:
import h5py
def load_cellreg(filename='pre_vs_post', keys=['pre', 'post']):
    with h5py.File('../data/%s.mat'%filename, 'r') as f:
        g = np.r_[f['cell_registered_struct']['cell_to_index_map']]
        cellreg_dict = {k:g[i][np.where(np.prod([gg>0 for gg in g], 0))[0]].astype(int)-1
                        for i, k in enumerate(keys)}
    return cellreg_dict

In [ ]:
#conditions = ['pre', 'post']
#cellregs = load_cellreg('pre_vs_post', conditions)

In [ ]:
import scipy.stats as sstats

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=3)

In [ ]:
vars_pre_traces_all =[]
for ani in pre_animals:
    vars_pre_traces_all.append(vars_pre['traces'][ani])
print vars_pre_traces_all
vars_pre_traces_all = reduce(lambda x,y: x+y,vars_pre_traces_all)
print vars_pre_traces_all


In [ ]:
traces_pca_pre = pca.fit_transform([vars_pre['traces'][ani] for ani in pre_animals])
traces_pca_post = pca.fit_transform([vars_post['traces'][ani] for ani in post_animals])
# traces_pca_pre = lle.fit_transform(vars_pre['traces'][:, cellregs['pre']])
# traces_pca_post = lle.transform(vars_post['traces'][:, cellregs['post']])


In [ ]:
def plot_mean_pca(ax, traces_pca_pre, vars_pre, odor='odor1', color='0.7', pca_dims=[0, 1], cmap=pl.cm.gray_r, tpre=4, tpost=4):
    all_traces = ut.extract_traces_around_event(vars_pre['time_ax'], traces_pca_pre, vars_pre['%s_ons'%odor],
                                                tpre=tpre, tpost=tpost)
    x, y = all_traces.mean(0)[:, pca_dims[0]], all_traces.mean(0)[:, pca_dims[1]]
    dx, dy = sstats.sem(all_traces, 0)[:, pca_dims[0]], sstats.sem(all_traces, 0)[:, pca_dims[1]]
    xyinit = x[0], y[0]
    ax.plot(x-xyinit[0], y-xyinit[1], '-', lw=0.5, color=color, zorder=2)
    ax.errorbar(x-xyinit[0], y-xyinit[1], dx, dy, elinewidth=0.5, zorder=1, color='0.7')
    colors = cmap(np.linspace(0.1, 0.9, all_traces.shape[1]))
    ax.scatter(x-xyinit[0], y-xyinit[1], s=2, linewidths=0, edgecolors=colors, c=colors, zorder=19)
    ax.set_xlabel('PC%d'%(pca_dims[0]+1), fontsize=5)
    ax.set_ylabel('PC%d'%(pca_dims[1]+1), fontsize=5)
    
fig, axs = pl.subplots(1, 2, figsize=(4, 2), sharex=True, sharey=True)
plot_mean_pca(axs[0], traces_pca_pre, vars_pre, odor='STIM1', pca_dims=[0, 1], tpre=0, tpost=15, color='red', cmap=pl.cm.Reds)
plot_mean_pca(axs[1], traces_pca_post, vars_post, odor='STIM1', pca_dims=[0, 1], tpre=0, tpost=15, color='red', cmap=pl.cm.Reds)

axs[0].set_title('CS+ 1, Pre', fontsize=7)
axs[1].set_title('CS+ 1, Post', fontsize=7)

lims = [ax.axis() for ax in axs]
xmin, xmax, ymin, ymax = [np.min([l[i] for l in lims]) for i in range(4)]
for ax in axs:
    pt.remove_axes(ax)
    ax.vlines(0, np.min((ymin, -ymax)), np.max((ymax, -ymin)), color='k', lw=0.5, zorder=0)
    ax.hlines(0, np.min((xmin, -xmax)), np.max((xmax, -xmin)), color='k', lw=0.5, zorder=0)


In [ ]:
def plot_mean_pca_3d(ax, traces_pca_pre, vars_pre, odor='odor1', color='0.7', pca_dims=[0, 1, 2],
                     cmap=pl.cm.gray_r, tpre=4, tpost=4, window=1):
    all_traces = ut.extract_traces_around_event(vars_pre['time_ax'], traces_pca_pre, vars_pre['%s_ons'%odor],
                                                tpre=tpre, tpost=tpost)
    conv_func = np.ones(window).astype(float)
    conv_func /= conv_func.sum()
    x = np.convolve(all_traces.mean(0)[:, pca_dims[0]], conv_func, mode='same')
    y = np.convolve(all_traces.mean(0)[:, pca_dims[1]], conv_func, mode='same')
    z = np.convolve(all_traces.mean(0)[:, pca_dims[2]], conv_func, mode='same')
    xyinit = x[0], y[0], z[0]
    ax.plot(x-xyinit[0], y-xyinit[1], z-xyinit[2], '-', lw=0.5, color=color, alpha=0.3)
    colors = cmap(np.linspace(0.1, 0.9, all_traces.shape[1]))
    ax.scatter(x-xyinit[0], y-xyinit[1], z-xyinit[2], s=5, linewidths=0, edgecolors=colors, c=colors, zorder=19)
    
    ax.plot([x[0]-xyinit[0]], [y[0]-xyinit[1]], [z[0]-xyinit[2]],
            'o', mew=1, mfc=(0, 0, 0, 0,), mec='k', ms=5, lw=0.5)
    
    ax.plot([x[-1]-xyinit[0]], [y[-1]-xyinit[1]], [z[-1]-xyinit[2]],
            'o', mew=1, mfc=(0, 0, 0, 0,), mec=color, ms=5, lw=0.5)
    
    pt.remove_axes(ax)
    ax.set_zticks(())
#     xmin, xmax, ymin, ymax = ax.axis()
#     ax.vlines(0, np.min((ymin, -ymax)), np.max((ymax, -ymin)), color='k', lw=0.5, zorder=0)
#     ax.hlines(0, np.min((xmin, -xmax)), np.max((xmax, -xmin)), color='k', lw=0.5, zorder=0)
    ax.set_xlabel('PC%d'%(pca_dims[0]+1), fontsize=5)
    ax.set_ylabel('PC%d'%(pca_dims[1]+1), fontsize=5)
    ax.set_zlabel('PC%d'%(pca_dims[2]+1), fontsize=5)
#     lims = np.max(abs(np.r_[ax.axis()]))
#     ax.set_xlim(-lims, lims)
#     ax.set_ylim(-lims, lims)
#     ax.set_zlim(-lims, lims)
    
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
view = (10, 10)

fig = pl.figure(figsize=(8, 2))

for i in range(4):
       
    ax = fig.add_subplot(1, 4, i+1, projection='3d')
    
    ax.set_title('Odor %d'%(i+1))
    
    ax.view_init(view[0], view[1])
    plot_mean_pca_3d(ax, traces_pca_pre, vars_pre, odor='STIM%d'%(i+1), pca_dims=[0, 1, 2], window=4,
                     tpre=0, tpost=5, color='grey', cmap=pl.cm.Greys)
#     ax.set_title('Pre', fontsize=7)
#     ax = fig.add_subplot(1, 4, i+1, projection='3d')
#     ax.view_init(view[0], view[1])
    plot_mean_pca_3d(ax, traces_pca_post, vars_post, odor='STIM%d'%(i+1), pca_dims=[0, 1, 2], window=4,
                     tpre=0, tpost=5, color='r', cmap=pl.cm.Reds)
#     ax.set_title('Post', fontsize=7)

    ax.plot([0], [0], [0], 'ko', mew=1, mfc=(0, 0, 0, 0,), mec='k', ms=5, lw=0.5)

    ax.set_xlim(-20, 20)
    ax.set_ylim(-20, 20)
    ax.set_zlim(-20, 20)
    
ax.legend([pt.Line2D([], [], lw=0.5, color='grey'),
           pt.Line2D([], [], lw=0.5, color='red')],
          ['Pre', 'Post'])

# def animate(i):
#     for ax in fig.axes:
#         ax.view_init(elev=10., azim=i*2)
#     return fig,

# from matplotlib import animation
# anim = animation.FuncAnimation(fig, animate,
#                                frames=50, interval=20, blit=True)
# # Save
# anim.save('basic_animation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])


In [ ]:
view = (10, 10)
tpost = 10

fig = pl.figure(figsize=(6, 6))


def plot_it(ax, t, v, o, cmap):

    ax.view_init(view[0], view[1])
    plot_mean_pca_3d(ax, t, v, odor=o, pca_dims=[0, 1, 2], window=4,
                     tpre=0, tpost=tpost, color='grey', cmap=cmap)

    ax.plot([0], [0], [0], 'ko', mew=1, mfc=(0, 0, 0, 0,), mec='k', ms=5, lw=0.5)

    ax.set_xlim(-20, 20)
    ax.set_ylim(-20, 20)
    ax.set_zlim(-20, 20)
    return ax

ax = fig.add_subplot(2, 2, 1, projection='3d')
plot_it(ax, traces_pca_pre, vars_pre, 'STIM1', pl.cm.Greens)
plot_it(ax, traces_pca_pre, vars_pre, 'STIM2', pl.cm.Reds)
ax.set_title("Pre", fontsize=7)
ax = fig.add_subplot(2, 2, 2, projection='3d')
plot_it(ax, traces_pca_post, vars_post, 'STIM1', pl.cm.Greens)
plot_it(ax, traces_pca_post, vars_post, 'STIM2', pl.cm.Reds)  
ax.set_title("Post", fontsize=7)

ax.legend([pt.Line2D([], [], lw=1, color='g'),
           pt.Line2D([], [], lw=1, color='r')],
          ['CS+1', 'CS+2'])

ax = fig.add_subplot(2, 2, 3, projection='3d')
plot_it(ax, traces_pca_pre, vars_pre, 'STIM3', pl.cm.Greens)
plot_it(ax, traces_pca_pre, vars_pre, 'STIM4', pl.cm.Reds)
ax = fig.add_subplot(2, 2, 4, projection='3d')
plot_it(ax, traces_pca_post, vars_post, 'STIM3', pl.cm.Greens)
plot_it(ax, traces_pca_post, vars_post, 'STIM4', pl.cm.Reds)
  
ax.legend([pt.Line2D([], [], lw=1, color='g'),
           pt.Line2D([], [], lw=1, color='r')],
          ['CS-1', 'CS-2'])

def animate(i):
    for ax in fig.axes:
        ax.view_init(elev=10., azim=i*2)
    return fig,

from matplotlib import animation
anim = animation.FuncAnimation(fig, animate,
                               frames=150, interval=20, blit=True)
# Save
anim.save('basic_animation_CS12.mp4', fps=30, extra_args=['-vcodec', 'libx264'])


In [ ]:
pl.plot(np.cumsum(pca.explained_variance_ratio_))
pl.ylabel('expl. var. ratio')
pl.xlabel('# components')

In [ ]:
save_workspace(db)